In [ ]:
import refnx
import numpy as np

from hogben.models.base import MagneticSLD
from hogben.models.samples import Sample
from hogben.optimise import Optimiser, optimise_parameters
from hogben.utils import Fisher
from hogben.visualise import angle_choice

In [ ]:
def protein_structure(ref_layer=False):
        # Slab thicknesses
        t_SiO2 = refnx.analysis.Parameter(
            15, 'SiO2 thickness', (0, 60)
        )
        t_inner_hg = refnx.analysis.Parameter(
            15, 'Inner headgroup thickness', (10, 30)
        )
        t_tg = refnx.analysis.Parameter(
            28, 'Total tailgroup thickness', (20, 40)
        )
        t_outer_hg = refnx.analysis.Parameter(
            18, 'Outer headgroup thickness', (10, 30)
        )
        t_extra = refnx.analysis.Parameter(
            60, 'Extramembrane domain thickness', (10, 35)
        )

        # Volume fractions and volumes
        phi_s = refnx.analysis.Parameter(
            0.36, 'Volume fraction of solvent', (0, 1)
        )
        phi_extra = refnx.analysis.Parameter(
            0.445, 'Extra membrane Molecular volume', (0, 1)
        )
        phi_innerhg = refnx.analysis.Parameter(
            0.47, 'Volume fraction solvent inner hg', (0.3, 0.7)
        )
        phi_outerhg = refnx.analysis.Parameter(
            0.83, 'Volume fraction solvent outer hg', (0.0, 1)
        )
        V_TMD = refnx.analysis.Parameter(
            0.245, 'TMD Molecular volume', (0, 0.6)
        )
        V_extra = refnx.analysis.Parameter(
            0.245, 'Extra membrane Molecular volume', (0, 0.6)
        )
        phi_tmd = phi_extra * (t_extra*V_TMD)/(t_tg*V_extra)

        # SLD Values
        sld_TMD = refnx.analysis.Parameter(
            2.5, 'TMD Protein SLD', (0, 25)
        )
        sld_lipacyl = refnx.analysis.Parameter(
            0.07, 'Lipid acyl chains tg SLD', (-1, 1)
        )
        sld_acyl = phi_tmd*sld_TMD + (1-phi_tmd)*sld_lipacyl
        sld_innerhg = refnx.analysis.Parameter(
            3, 'SLD inner hg', (1, 5)
        )
        sld_outerhg = refnx.analysis.Parameter(
            3, 'SLD outer hg', (1, 5)
        )
        sld_extra_h = refnx.analysis.Parameter(
            1.8, 'SLD extramembrane in H2O', (0, 5)
        )
        sld_extra_d = refnx.analysis.Parameter(
            2.9, 'SLD extramembrane in D2O', (0, 5)
        )
        sld_extra_smw = refnx.analysis.Parameter(
            2.2, 'SLD extramembrane in SMW', (0, 5)
        )
        sld_sio2 = refnx.analysis.Parameter(
            4.2, 'SLD sio2', (0, 10)
        )
        sld_h2o = refnx.analysis.Parameter(
            -0.56, 'Contrast 1', (-0.56, 6.14)
        )
        sld_smw = refnx.analysis.Parameter(
            2.07, 'Contrast 2', (-0.56, 6.14)
        )
        sld_d2o = refnx.analysis.Parameter(
            6.14, 'Contrast 3', (-0.56, 6.14)
        )

        # Roughness
        r_SiO2 = refnx.analysis.Parameter(
            0, 'SiO2 roughness', (0, 15)
        )
        r_inner_hg = refnx.analysis.Parameter(
            0, 'Inner headgroup roughness', (1, 5)
        )
        r_tg = refnx.analysis.Parameter(
            0, 'Total tailgroup roughness', (1, 5)
        )
        r_outer_hg = refnx.analysis.Parameter(
            0, 'Outer headgroup roughness', (1, 5)
        )
        r_extra = refnx.analysis.Parameter(
            0, 'Extramembrane domain roughness', (1, 5)
        )
        
        # DEfine slabs
        sub = refnx.reflect.SLD(2.074)
        SiO2 = refnx.reflect.SLD(sld_sio2, name="SiO2")(t_SiO2,
                                                r_SiO2)

        inner_hg = refnx.reflect.SLD(sld_innerhg, name="Inner Head Group")(t_inner_hg,
                                                       r_inner_hg,
                                                       vfsolv=phi_innerhg)
        tail = refnx.reflect.SLD(sld_acyl, name="Tail groups")(t_tg,
                                                r_tg,
                                                vfsolv=phi_s)
        outer_hg = refnx.reflect.SLD(sld_outerhg, name = "Outer head group")(t_outer_hg,
                                                       r_outer_hg,
                                                       vfsolv=phi_outerhg)
        extra_d = refnx.reflect.SLD(sld_extra_d, "Extra protein")(t_extra,
                                                  r_extra,
                                                  vfsolv=0)
        extra_h = refnx.reflect.SLD(sld_extra_h, "Extra protein")(t_extra,
                                                  r_extra,
                                                  vfsolv=0)
        extra_smw = refnx.reflect.SLD(sld_extra_smw, "Extra protein")(t_extra,
                                                  r_extra,
                                                  vfsolv=0)
        
        # Define solvents
        H2O = refnx.reflect.SLD(sld_h2o, name="H2O solvent")(rough=0)
        D2O = refnx.reflect.SLD(sld_d2o, name="D2O solvent")(rough=0)
        SMW = refnx.reflect.SLD(sld_smw, name="SMW solvent")(rough=0)

        # Choose varying parameters
        params = [
     #       t_tg,
            # t_SiO2,
     #       t_extra,
     #       t_outer_hg,
     #       t_inner_hg,
     #        phi_innerhg,
     #        phi_outerhg,
            phi_s,
      #      phi_extra,
            # r_tg,
            # r_inner_hg,
            # r_outer_hg,
         #   sld_outerhg,
         #   sld_lipacyl,
        #    sld_innerhg,
       #     sld_extra_d,
        #    sld_extra_smw,
     #       sld_extra_h
        ]

        for param in params:
            param.vary = True
        
        # Define reference layer
        ul_thick = refnx.analysis.Parameter(50, 'Underlayer Thickness', (1e-7, 250))
        ul_nsld = refnx.analysis.Parameter(5, 'Underlayer Nuclear SLD', (0.5, 8))
        ul_msld = refnx.analysis.Parameter(2.7, 'Underlayer Magnetic SLD', (1e-7, 6))
        ul_thick.optimize = True
        ul_nsld.optimize = True
        ul_msld.optimize = True
        
        # Magnetic reference layer
        ul_slab = MagneticSLD(SLDn=ul_nsld, SLDm=ul_msld)(thick=ul_thick, rough=0)

        if ref_layer:
            # With reference layer                    
            structure_d2o = D2O | extra_d | outer_hg | tail | inner_hg | ul_slab | SiO2 | sub
            structure_SMW = SMW | extra_d | outer_hg | tail | inner_hg | ul_slab | SiO2 | sub
            structure_h2o = H2O | extra_d | outer_hg | tail | inner_hg | ul_slab | SiO2 | sub        
            
        else:
            structure_d2o = D2O | extra_d | outer_hg | tail | inner_hg | SiO2 | sub
            structure_SMW = SMW | extra_d | outer_hg | tail | inner_hg | SiO2 | sub
            structure_h2o = H2O | extra_d | outer_hg | tail | inner_hg | SiO2 | sub        
            
        


        structures = [
                           structure_d2o,
          #                structure_SMW,
                           structure_h2o,
                           ]
        return structures

# Define samples

In [ ]:
new_sample_ref = Sample(protein_structure(ref_layer=True))
new_sample_ref.polarised = False # Set to false to make it nuclear ref layer
new_sample_ref.sld_profile("")

new_sample = Sample(protein_structure(ref_layer=False))
new_sample.sld_profile("")


# Optimise reference layer

In [ ]:
angle_times = [(0.7, 100, 25), (2.0, 100, 75)]
optimise_parameters(new_sample_ref, angle_times)

# Optimise angle_times

In [ ]:
optimiser = Optimiser(new_sample_ref)
num_angles = 2 # Number of angles to optimise.

# Find the optimal angles and times using differential evolution.
angles, splits, _ = optimiser.optimise_angle_times(num_angles, angle_bounds=(0.2, 2.3), verbose=False)

# Calculate the counting times from the splits using total counting time of 100.
times = np.array(splits) * 100

print('Angles: {}'.format(np.round(angles, 2)))
print('Times: {}'.format(np.round(times, 1)))

# Get improvement compared to unoptimised (no ref. layer, unoptimal angle_times)

In [ ]:
angle_times_optimised = [(angles[0], 100, times[0]), (angles[1], 100, times[1])]
fisher_optimised = Fisher.from_sample(new_sample_ref, angle_times_optimised)
print(fisher_optimised.min_eigenval)

angle_times_unoptimised = [(0.7, 100, 25), (2.0, 100, 75)]
fisher_unoptimised = Fisher.from_sample(new_sample, angle_times_unoptimised)
print(fisher_unoptimised.min_eigenval)
improvement = fisher_optimised.min_eigenval/fisher_unoptimised.min_eigenval
if new_sample_ref.is_magnetic() and new_sample.polarised:
    improvement /= 4
print(f'Optimising gave an improvement of a factor {round(improvement,2)}')

# Visualising

In [ ]:
# Path to directory to save results to.
save_path = '../examples/results'

# Number of data points to simulate.
points = 150

# Time to use for simulation.
# A time of 1 approximately corresponds to 1.5 minutes.
time = 40

# Angles (in degrees) to consider.
angle_range = np.linspace(0.2, 2.3, 200)
initial_angle_times = [] # No angles measured so far.
angle_1 = angle_choice(new_sample, initial_angle_times, angle_range, points, time, save_path, 'angle_choice_1')
print('First angle choice: {}'.format(round(angle_1, 2)))


In [ ]:
x = 6 # Change x to adjust the counting time.
initial_angle_times = [(angle_1, points, time*x)] # (Angle, Points, Time)
angle_2 = angle_choice(new_sample, initial_angle_times, angle_range, points, time, save_path, 'angle_choice_2')
print('Second angle choice: {}'.format(round(angle_2, 2)))